目录

一、str对象

    1. str对象的设计意图
    2. []索引器
    3. string类型

二、正则表达式基础

    1. 一般字符的匹配
    2. 元字符基础
    3. 简写字符集

三、文本处理的五类操作

    1. 拆分
    2. 合并
    3. 匹配
    4. 替换
    5. 提取

四、常用字符串函数

    1. 字母型函数
    2. 数值型函数
    3. 统计型函数
    4. 格式型函数

五、练习

<center><h1>第八章 文本数据</h1></center>

In [1]:
import numpy as np
import pandas as pd

## 一、str对象
### 1. str对象的设计意图


In [2]:
#str对象是定义在Index或Series上的属性，专门用于处理每个元素的文本内容，其内部定义了大量方法，因此对一个序列进行文本处理，首先需要获取其str对象。
#在pandas的50个str对象方法中，有31个是和标准库中的str模块方法同名且功能一致

#标准库中的str模块方法:字母转为大写的操作
var = 'abcd'
str.upper(var) # Python内置str模块

'ABCD'

In [3]:
s = pd.Series(['abcd', 'efg', 'hi'])
s.str

In [4]:
s.str.upper() # pandas中str对象上的upper方法

0    ABCD
1     EFG
2      HI
dtype: object

### 2. []索引器

In [5]:
#对于str对象而言，可理解为其对字符串进行了序列化的操作

#通过[]可以取出某个位置的元素
var[0]

'a'

In [6]:
#切片得到子串
var[-1: 0: -2]

'db'

In [7]:
#通过对str对象使用[]索引器，可以完成完全一致的功能，并且如果超出范围则返回缺失值

s.str[0]

0    a
1    e
2    h
dtype: object

In [8]:
s.str[-1: 0: -2]

0    db
1     g
2     i
dtype: object

In [9]:
s.str[2]

0      c
1      g
2    NaN
dtype: object

### 3. string类型


In [18]:
#object类型只应当存储混合类型，例如同时存储浮点、字符串、字典、列表、自定义类型等
s = pd.Series([{1: 'temp_1', 2: 'temp_2'}, ['a', 'b'], 0.5, 'my_string'])
s

0    {1: 'temp_1', 2: 'temp_2'}
1                        [a, b]
2                           0.5
3                     my_string
dtype: object

In [19]:
s.str[1]
# 0: 1的值是temp_1
# 1: 0的值是a，1的值是b
# 2: 0的值是0.5
# 3: 0的值是m，1的值是y

0    temp_1
1         b
2       NaN
3         y
dtype: object

In [22]:
#string类型
s.astype('string')

0    {1: 'temp_1', 2: 'temp_2'}
1                    ['a', 'b']
2                           0.5
3                     my_string
dtype: string

In [23]:
s.astype('string').str[1]
# 0: 0的值是{     1的值是 1
# 1: 0的值是[     1的值是 '
# 2: 0的值是0     1的值是 .
# 3: 0的值是m     1的值是 y

0    1
1    '
2    .
3    y
dtype: string

`string`类型是`Nullable`类型，但`object`不是。这意味着`string`类型的序列，如果调用的`str`方法返回值为整数`Series`和布尔`Series`时，其分别对应的`dtype`是`Int`和`boolean`的`Nullable`类型，而`object`类型则会分别返回`int/float`和`bool/object`，取决于缺失值的存在与否。同时，字符串的比较操作，也具有相似的特性，`string`返回`Nullable`类型，但`object`不会。

In [24]:
s = pd.Series(['a'])
s.str.len()

0    1
dtype: int64

In [25]:
s.astype('string').str.len()

0    1
dtype: Int64

In [26]:
s == 'a'

0    True
dtype: bool

In [27]:
s.astype('string') == 'a'

0    True
dtype: boolean

In [28]:
s = pd.Series(['a', np.nan]) # 带有缺失值

In [29]:
s.str.len()

0    1.0
1    NaN
dtype: float64

In [30]:
s.astype('string').str.len()

0       1
1    <NA>
dtype: Int64

In [31]:
s == 'a'

0     True
1    False
dtype: bool

In [32]:
s.astype('string') == 'a'

0    True
1    <NA>
dtype: boolean

In [34]:
#对于全体元素为数值类型的序列，即使其类型为object或者category也不允许直接使用str属性
s = pd.Series([12, 345, 6789])
s.str[1]

AttributeError: Can only use .str accessor with string values!

In [35]:
#如果需要把数字当成string类型处理，可以使用astype强制转换为string类型的Series
s = pd.Series([12, 345, 6789])
s.astype('string').str[1]

0    2
1    4
2    7
dtype: string

## 二、正则表达式基础

[learn-regex-zh](https://github.com/cdoco/learn-regex-zh)

[正则表达式必知必会](https://book.douban.com/subject/26285406/)

### 1. 一般字符的匹配


In [36]:
#第一个参数是正则表达式，第二个参数是待匹配的字符串
#在下面的字符串中找出apple
import re
re.findall(r'Apple', 'Apple! This Is an Apple!')

['Apple', 'Apple']

### 2. 元字符基础
|元字符 |   描述 |
| :-----| ----: |
|.       |    匹配除换行符以外的任意字符|
|\[ \]     |      字符类，匹配方括号中包含的任意字符|
|\[^ \]     |      否定字符类，匹配方括号中不包含的任意字符|
|\*       |    匹配前面的子表达式零次或多次|
|\+       |    匹配前面的子表达式一次或多次|
|?        |   匹配前面的子表达式零次或一次|
|{n,m}    |       花括号，匹配前面字符至少 n 次，但是不超过 m 次|
|(xyz)   |        字符组，按照确切的顺序匹配字符xyz|
|\|     |      分支结构，匹配符号之前的字符或后面的字符|
|\\    |       转义符，它可以还原元字符原来的含义|
|^    |       匹配行的开始|
|$   |        匹配行的结束|

In [37]:
# . 匹配除换行符以外的任意字符
re.findall(r'.', 'abc')

['a', 'b', 'c']

In [38]:
# [ ] 字符类，匹配方括号中包含的任意字符
re.findall(r'[ac]', 'abc')

['a', 'c']

In [39]:
# [^ ] 否定字符类，匹配方括号中不包含的任意字符
re.findall(r'[^ac]', 'abc')

['b']

In [46]:
# {n,m} 花括号，匹配前面字符至少 n 次，但是不超过 m 次
re.findall(r'[ab]{2}', 'aaaabbbb') # {n}指匹配n次

['aa', 'aa', 'bb', 'bb']

In [47]:
# | 分支结构，匹配符号之前的字符或后面的字符
re.findall(r'aaa|bbb', 'aaaabbbb')

['aaa', 'bbb']

In [48]:
# (xyz) 字符组，按照确切的顺序匹配字符xyz
re.findall(r'(ab)', 'aaaabbbb') 

['ab']

In [54]:
# * 匹配前面的子表达式零次或多次
re.findall(r'a*', 'aa?a*aaa')

['aa', '', 'a', '', 'aaa', '']

In [55]:
# + 匹配前面的子表达式一次或多次
re.findall(r'a+', 'aa?a*aaa')

['aa', 'a', 'aaa']

In [56]:
# ? 匹配前面的子表达式零次或一次
re.findall(r'a?', 'aa?a*a')

['a', 'a', '', 'a', '', 'a', '']

In [59]:
# \ 转义符，它可以还原元字符原来的含义
re.findall(r'a\?', 'a?a?a*a')

['a?', 'a?']

In [60]:

re.findall(r'a\\?|a\*', 'aa?a*a')

['a', 'a', 'a', 'a']

In [61]:
re.findall(r'a?.', 'abaacadaae')

['ab', 'aa', 'c', 'ad', 'aa', 'e']

### 3. 简写字符集
此外，正则表达式中还有一类简写字符集，其等价于一组字符的集合：

|简写    |  描述 |
| :-----| :---- |
|\\w     |   匹配所有字母、数字、下划线: \[a-zA-Z0-9\_\] |
|\\W     |   匹配非字母和数字的字符: \[^\\w\]|
|\\d     |   匹配数字: \[0-9\]|
|\\D   |     匹配非数字: \[^\\d\]|
|\\s    |    匹配空格符: \[\\t\\n\\f\\r\\p{Z}\]|
|\\S    |    匹配非空格符: \[^\\s\]|
|\\B  |      匹配一组非空字符开头或结尾的位置，不代表具体字符|

In [78]:
# \w 匹配所有字母、数字、下划线
re.findall(r'\w{2}', '09 8? 7w c_ 9q p@')

['09', '7w', 'c_', '9q']

In [84]:
# \W 匹配非字母和数字的字符
re.findall(r'\W{2}', '09 8? 7w c_ 9q p@ 1')

['? ', '@ ']

In [85]:
# \d 匹配数字: [0-9]
re.findall(r'\d{2}', '09 8? 7w c_ 9q p@')

['09']

In [86]:
# \D 匹配非数字: [^\d]
re.findall(r'\D{2}', '09 8? 7w c_ 9q p@')

['? ', 'w ', 'c_', 'q ', 'p@']

In [69]:
re.findall(r'.s', 'Apple! This Is an Apple!')

['is', 'Is']

In [72]:
# \s 匹配空格符
re.findall(r'.\s.', 'Constant dropping wears the stone.')

['t d', 'g w', 's t', 'e s']

In [75]:
# \S 匹配非空格符
re.findall(r'.\S.', 'Constant dropping wears the stone.')

['Con', 'sta', 'nt ', 'dro', 'ppi', 'ng ', 'wea', 'rs ', 'the', ' st', 'one']

In [89]:
# \B	匹配一组非空字符开头或结尾的位置，不代表具体字符
re.findall(r'\B', 'Con')

['', '']

In [90]:
re.findall(r'\B', 'Cons')

['', '', '']

In [92]:
#匹配第一个位置是数字、字母、下划线
#第二个位置是非数字、非字母
re.findall(r'\w\W', '09 8? 7w c_ 9q p@')

['9 ', '8?', 'w ', '_ ', 'q ', 'p@']

In [93]:
#上一个条件下，不是以空开头或结尾
re.findall(r'\w\W\B', '09 8? 7w c_ 9q p@')

['8?', 'p@']

In [95]:
# 匹配某区，某路，某号，输出加上（）
re.findall(r'上海市(.{2,3}区)(.{2,3}路)(\d+号)', '上海市黄浦区方浜中路249号 上海市宝山区密山路5号')

[('黄浦区', '方浜中路', '249号'), ('宝山区', '密山路', '5号')]

## 三、文本处理的五类操作
### 1. 拆分

`str.split`能够把字符串的列进行拆分，其中第一个参数为正则表达式，可选参数包括从左到右的最大拆分次数`n`，是否展开为多个列`expand`。

In [96]:
# str.split能够把字符串的列进行拆分
s = pd.Series(['上海市黄浦区方浜中路249号', '上海市宝山区密山路5号'])
s.str.split('[市区路]')

0    [上海, 黄浦, 方浜中, 249号]
1       [上海, 宝山, 密山, 5号]
dtype: object

In [97]:
# 第一个参数为正则表达式，可选参数包括从左到右的最大拆分次数n，是否展开为多个列expand
s.str.split('[市区路]', n=2, expand=True)

,0,1,2
0,上海,黄浦,方浜中路249号
1,上海,宝山,密山路5号


In [98]:
s.str.split('[市区路]', n=1, expand=True)

,0,1
0,上海,黄浦区方浜中路249号
1,上海,宝山区密山路5号


与其类似的函数是`str.rsplit`，其区别在于使用`n`参数的时候是从右到左限制最大拆分次数。但是当前版本下`rsplit`因为`bug`而无法使用正则表达式进行分割：

In [99]:
# 与其类似的函数是str.rsplit，其区别在于使用n参数的时候是从右到左限制最大拆分次数。
# 但是当前版本下rsplit因为bug而无法使用正则表达式进行分割
s.str.rsplit('[市区路]', n=2, expand=True)

,0
0,上海市黄浦区方浜中路249号
1,上海市宝山区密山路5号


### 2. 合并


In [102]:
# str.join表示用某个连接符把Series中的字符串列表连接起来，如果列表中出现了非字符串元素则返回缺失值
s = pd.Series([['a','b'], [1, 'a'], [['a', 'b'], 'c']])
s.str.join('-')

0    a-b
1    NaN
2    NaN
dtype: object

In [103]:
# str.cat用于合并两个序列，主要参数为连接符sep、连接形式join以及缺失值替代符号na_rep，其中连接形式默认为以索引为键的左连接。
s1 = pd.Series(['a','b'])
s2 = pd.Series(['cat','dog'])
s1.str.cat(s2,sep='-')

0    a-cat
1    b-dog
dtype: object

In [104]:
s2.index = [1, 2]
s1.str.cat(s2, sep='-', na_rep='?', join='outer')

0      a-?
1    b-cat
2    ?-dog
dtype: object

### 3. 匹配

`str.contains`返回了每个字符串是否包含正则模式的布尔序列：

In [108]:
re.findall(r'\s\wat', 'my cat  he is fat  railway station')

[' cat', ' fat']

In [109]:
s = pd.Series(['my cat', 'he is fat', 'railway station'])
s.str.contains('\s\wat')

0     True
1     True
2    False
dtype: bool

In [110]:
# str.startswith返回了每个字符串以给定模式为开始的布尔序列，不支持正则表达式：
s.str.startswith('my')

0     True
1    False
2    False
dtype: bool

In [111]:
# str.endswith返回了每个字符串以给定模式为结束的布尔序列，不支持正则表达式：
s.str.endswith('t')

0     True
1     True
2    False
dtype: bool

In [112]:
# 用正则表达式来检测开始或结束字符串的模式，可以使用str.match，其返回了每个字符串起始处是否符合给定正则模式的布尔序列
s.str.match('m|h')

0     True
1     True
2    False
dtype: bool

In [113]:
s.str[::-1].str.match('ta[f|g]|n') # 反转后匹配

0    False
1     True
2     True
dtype: bool

In [114]:
# 这些也能通过在str.contains的正则中使用^和$来实现
s.str.contains('^[m|h]')

0     True
1     True
2    False
dtype: bool

In [115]:
s.str.contains('[f|g]at|n$')

0    False
1     True
2     True
dtype: bool

In [116]:
# 返回从左到右第一次匹配的位置的索引，未找到则返回-1。
# 不支持正则匹配，只能用于字符子串的匹配
s = pd.Series(['This is an apple. That is not an apple.'])
s.str.find('apple')

0    11
dtype: int64

In [117]:
# 返回从右到左第一次匹配的位置的索引，未找到则返回-1。
# 不支持正则匹配，只能用于字符子串的匹配
s.str.rfind('apple')

0    33
dtype: int64

### 4. 替换


In [118]:
# str.replace和replace并不是一个函数，在使用字符串替换时应当使用前者。
s = pd.Series(['a_1_b','c_h'])
s.str.replace('\d|\?', 'new', regex=True)

0    a_new_b
1      c_new
dtype: object

In [119]:
# 利用子组的方法对不同部分进行有差别的替换
# 可以通过传入自定义的替换函数来分别进行处理

s = pd.Series(['上海市黄浦区方浜中路249号',
                '上海市宝山区密山路5号',
                '北京市昌平区北农路2号'])
pat = '(\w+市)(\w+区)(\w+路)(\d+号)' # \w数字字母下划线  \d数字  +匹配前面的一次或多次
city = {'上海市': 'Shanghai', '北京市': 'Beijing'}
district = {'昌平区': 'CP District',
            '黄浦区': 'HP District',
            '宝山区': 'BS District'}
road = {'方浜中路': 'Mid Fangbin Road',
        '密山路': 'Mishan Road',
        '北农路': 'Beinong Road'}
def my_func(m):
    str_city = city[m.group(1)] # group(k)代表匹配到的第k个子组（圆括号之间的内容）
    str_district = district[m.group(2)]
    str_road = road[m.group(3)]
    str_no = 'No. ' + m.group(4)[:-1]
    return ' '.join([str_city,
                     str_district,
                     str_road,
                     str_no])
s.str.replace(pat, my_func, regex=True)

0    Shanghai HP District Mid Fangbin Road No. 249
1           Shanghai BS District Mishan Road No. 5
2           Beijing CP District Beinong Road No. 2
dtype: object

In [120]:
# 可以使用命名子组更加清晰地写出子组代表的含义

pat = '(?P<市名>\w+市)(?P<区名>\w+区)(?P<路名>\w+路)(?P<编号>\d+号)'
def my_func(m):
    str_city = city[m.group('市名')]
    str_district = district[m.group('区名')]
    str_road = road[m.group('路名')]
    str_no = 'No. ' + m.group('编号')[:-1]
    return ' '.join([str_city,
                     str_district,
                     str_road,
                     str_no])
s.str.replace(pat, my_func, regex=True)

0    Shanghai HP District Mid Fangbin Road No. 249
1           Shanghai BS District Mishan Road No. 5
2           Beijing CP District Beinong Road No. 2
dtype: object


### 5. 提取


In [121]:
# 提取既可以认为是一种返回具体元素的匹配操作，也可以认为是一种特殊的拆分操作。
# str.split例子中会把分隔符去除，这时候就可以用str.extract进行提取

pat = '(\w+市)(\w+区)(\w+路)(\d+号)'
s.str.extract(pat)

,0,1,2,3
0,上海市,黄浦区,方浜中路,249号
1,上海市,宝山区,密山路,5号
2,北京市,昌平区,北农路,2号


In [122]:
# 通过子组的命名，可以直接对新生成DataFrame的列命名

pat = '(?P<市名>\w+市)(?P<区名>\w+区)(?P<路名>\w+路)(?P<编号>\d+号)'
s.str.extract(pat)

,市名,区名,路名,编号
0,上海市,黄浦区,方浜中路,249号
1,上海市,宝山区,密山路,5号
2,北京市,昌平区,北农路,2号


In [123]:
# str.extract只匹配一次
# str.extractall会把所有符合条件的模式全部匹配出来，如果存在多个结果，则以多级索引的方式存储

s = pd.Series(['A135T15,A26S5','B674S2,B25T6'], index = ['my_A','my_B'])
pat = '[A|B](\d+)[T|S](\d+)'
s.str.extractall(pat)

0   1
     match         
my_A 0      135  15
     1       26   5
my_B 0      674   2
     1       25   6

In [124]:
# 通过子组的命名，可以直接对新生成DataFrame的列命名
pat_with_name = '[A|B](?P<name1>\d+)[T|S](?P<name2>\d+)'
s.str.extractall(pat_with_name)

name1 name2
     match            
my_A 0       135    15
     1        26     5
my_B 0       674     2
     1        25     6

`str.findall`的功能类似于`str.extractall`，区别在于前者把结果存入列表中，而后者处理为多级索引，每个行只对应一组匹配，而不是把所有匹配组合构成列表。

In [125]:
# str.findall的功能类似于str.extractall
# str.findall把结果存入列表中
# str.extractal把结果处理为多级索引，每个行只对应一组匹配

s.str.findall(pat)

my_A    [(135, 15), (26, 5)]
my_B     [(674, 2), (25, 6)]
dtype: object

## 四、常用字符串函数


### 1. 字母型函数


In [126]:
s = pd.Series(['lower', 'CAPITALS', 'this is a sentence', 'SwApCaSe'])
s.str.upper() # 转为大写

0                 LOWER
1              CAPITALS
2    THIS IS A SENTENCE
3              SWAPCASE
dtype: object

In [127]:
s.str.lower() # 转为小写

0                 lower
1              capitals
2    this is a sentence
3              swapcase
dtype: object

In [128]:
s.str.title() # 单词以大写开头

0                 Lower
1              Capitals
2    This Is A Sentence
3              Swapcase
dtype: object

In [129]:
s.str.capitalize() # 首字母转为大写，其余小写

0                 Lower
1              Capitals
2    This is a sentence
3              Swapcase
dtype: object

In [130]:
s.str.swapcase() # 大写转小写，小写转大写

0                 LOWER
1              capitals
2    THIS IS A SENTENCE
3              sWaPcAsE
dtype: object

### 2. 数值型函数


In [131]:
# pd.to_numeric主要参数包括errors（非数值的处理模式）和downcast（转换类型）
# 有三种errors选项，raise（直接报错）, coerce（设为缺失）, ignore（保持原来的字符串）

s = pd.Series(['1', '2.2', '2e', '??', '-2.1', '0'])
pd.to_numeric(s, errors='ignore')

0       1
1     2.2
2      2e
3      ??
4    -2.1
5       0
dtype: object

In [132]:
pd.to_numeric(s, errors='coerce')

0    1.0
1    2.2
2    NaN
3    NaN
4   -2.1
5    0.0
dtype: float64

In [133]:
pd.to_numeric(s, errors='raise')

ValueError: Unable to parse string "2e" at position 2

In [65]:
# 在数据清洗时，可以利用coerce的设定，快速查看非数值型的行

s[pd.to_numeric(s, errors='coerce').isna()]

2    2e
3    ??
dtype: object

### 3. 统计型函数

In [134]:
# count返回出现正则模式的次数
s = pd.Series(['cat rat fat at', 'get feed sheet heat'])
s.str.count('[r|f]at|ee')

0    2
1    2
dtype: int64

In [135]:
# len返回出现正则模式的字符串的长度
s.str.len()

0    14
1    19
dtype: int64

### 4. 格式型函数
格式型函数主要分为两类，第一种是除空型，第二种是填充型。

In [136]:
# 除空型在数据清洗时是有用的，特别是列名含有非法空格的时候
# strip去除两侧空格
my_index = pd.Index([' col1', 'col2 ', ' col3 '])
my_index.str.strip().str.len()

Int64Index([4, 4, 4], dtype='int64')

In [137]:
# rstrip去除右侧空格
my_index.str.rstrip().str.len()

Int64Index([5, 4, 5], dtype='int64')

In [138]:
# lstrip去除左侧空格
my_index.str.lstrip().str.len()

Int64Index([4, 5, 5], dtype='int64')

In [139]:
# 填充型函数，pad是最灵活的，它可以选定字符串长度、填充的方向和填充内容

s = pd.Series(['a','b','c'])
s.str.pad(5,'left','*')

0    ****a
1    ****b
2    ****c
dtype: object

In [140]:
s.str.pad(5,'right','*')

0    a****
1    b****
2    c****
dtype: object

In [141]:
s.str.pad(5,'both','*')

0    **a**
1    **b**
2    **c**
dtype: object

In [142]:
# 与s.str.pad(5,'left','*')等效
s.str.rjust(5, '*')

0    ****a
1    ****b
2    ****c
dtype: object

In [143]:
# 与s.str.pad(5,'right','*')等效
s.str.ljust(5, '*')

0    a****
1    b****
2    c****
dtype: object

In [144]:
# 与s.str.pad(5,'both','*')等效
s.str.center(5, '*')

0    **a**
1    **b**
2    **c**
dtype: object

In [145]:
# 在读取excel文件时，经常会出现数字前补0的需求，例如证券代码读入的时候会把"000007"作为数值7来处理
# 可以使用上面的左侧填充函数
s = pd.Series([7, 155, 303000]).astype('string')
s.str.pad(6,'left','0')

0    000007
1    000155
2    303000
dtype: string

In [146]:
s.str.rjust(6,'0')

0    000007
1    000155
2    303000
dtype: string

In [147]:
# 用zfill来实现
s.str.zfill(6)

0    000007
1    000155
2    303000
dtype: string

## 五、练习
### Ex1：房屋信息数据集
现有一份房屋信息数据集如下：

In [148]:
df = pd.read_excel('./data/house_info.xls', usecols=['floor','year','area','price'])
df.head(3)

,floor,year,area,price
0,高层（共6层）,1986年建,58.23㎡,155万
1,中层（共20层）,2020年建,88㎡,155万
2,低层（共28层）,2010年建,89.33㎡,365万


In [149]:
# 1. 将`year`列改为整数年份存储。
df = pd.read_excel('data/house_info.xls', usecols=['floor','year','area','price'])

df.year = pd.to_numeric(df.year.str[:-2]).astype('Int64')

df.head(3)

,floor,year,area,price
0,高层（共6层）,1986,58.23㎡,155万
1,中层（共20层）,2020,88㎡,155万
2,低层（共28层）,2010,89.33㎡,365万


In [150]:
# 2. 将`floor`列替换为`Level, Highest`两列，其中的元素分别为`string`类型的层类别（高层、中层、低层）与整数类型的最高层数。
pat = '(\w层)（共(\d+)层）'

new_cols = df.floor.str.extract(pat).rename(columns={0:'Level', 1:'Highest'})

df = pd.concat([df.drop(columns=['floor']), new_cols], 1)

df.head(3)

,year,area,price,Level,Highest
0,1986,58.23㎡,155万,高层,6
1,2020,88㎡,155万,中层,20
2,2010,89.33㎡,365万,低层,28


In [151]:
# 3. 计算房屋每平米的均价`avg_price`，以`***元/平米`的格式存储到表中，其中`***`为整数。
s_area = pd.to_numeric(df.area.str[:-1])

s_price = pd.to_numeric(df.price.str[:-1])

df['avg_price'] = ((s_price/s_area)*10000).astype('int').astype('string') + '元/平米'


df.head(3)

,year,area,price,Level,Highest,avg_price
0,1986,58.23㎡,155万,高层,6,26618元/平米
1,2020,88㎡,155万,中层,20,17613元/平米
2,2010,89.33㎡,365万,低层,28,40859元/平米



### Ex2：《权力的游戏》剧本数据集
现有一份权力的游戏剧本数据集如下：

In [152]:
df = pd.read_csv('./data/script.csv')
df.head(3)

,Release Date,Season,Episode,Episode Title,Name,Sentence
0,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,What do you expect? They're savages. One lot s...
1,2011-04-17,Season 1,Episode 1,Winter is Coming,will,I've never seen wildlings do a thing like this...
2,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,How close did you get?


In [153]:
# 1. 计算每一个`Episode`的台词条数。
df = pd.read_csv('data/script.csv')

df.columns = df.columns.str.strip()

df.groupby(['Season', 'Episode'])['Sentence'].count().head()

Season    Episode   
Season 1  Episode 1     327
          Episode 10    266
          Episode 2     283
          Episode 3     353
          Episode 4     404
Name: Sentence, dtype: int64

In [154]:
# 2. 以空格为单词的分割符号，请求出单句台词平均单词量最多的前五个人。
df.set_index('Name').Sentence.str.split().str.len().groupby('Name').mean().sort_values(ascending=False).head()


Name
male singer          109.000000
slave owner           77.000000
manderly              62.000000
lollys stokeworth     62.000000
dothraki matron       56.666667
Name: Sentence, dtype: float64

In [155]:
# 3. 若某人的台词中含有问号，那么下一个说台词的人即为回答者。若上一人台词中含有$n$个问号，则认为回答者回答了$n$个问题，请求出回答最多问题的前五个人。
s = pd.Series(df.Sentence.values, index=df.Name.shift(-1))

s.str.count('\?').groupby('Name').sum().sort_values(ascending=False).head()

Name
tyrion lannister    527
jon snow            374
jaime lannister     283
arya stark          265
cersei lannister    246
dtype: int64